In [1]:
# 引入库
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.preprocessing import StandardScaler

In [2]:
# 检查os位置
print(os.getcwd())

/Users/wolves/Downloads/project/python/pt/lab


生成数据方式
$$y = 1.35x + 0.75x^2 + 2.1\sqrt{x} + 2.89$$

In [3]:
# 生成数据
def generate_data():
    w = np.array([1.35, 0.75, 2.1])  # 权重
    b = 2.89  # 偏置
    x_min = 1
    x_max = 8
    x = np.linspace(x_min, x_max, 10) # 均匀分布
    X = np.array([x, x**2, np.sqrt(x)])  # 特征矩阵3x10
    y = np.dot(w, X) + b # 1x10 一维向量不区分行向量和列向量
    y += np.random.normal(scale=0.5, size=y.shape)
    data = np.column_stack((X.T, y)) # 10x4
    scaler = StandardScaler()
    data = scaler.fit_transform(data)
    return data

# 保存数据
def save_data(filename, data):
    np.savetxt(filename, data, delimiter=',')
    print(f"{filename} 已成功创建并写入数据。")

# 生成并保存数据
data = generate_data()
#save_data('./1_data.txt', data)

In [4]:
# 读取数据
#points = np.genfromtxt("./1_data.txt", delimiter=',')

points = data

m = len(points[:,0])
x = points[:, :3] # 10x3
y = points[:,3] # 1x10

原函数：
$$
\vec{w} = {\begin{bmatrix} w_1 & w_2 & w_3 & \cdots & w_n \end{bmatrix}}^T
$$

$$
\vec{x} = \begin{bmatrix} x_1 & x_2 & x_3 & \cdots & x_n \end{bmatrix}
$$

$$
f_{\vec{w} \cdot,b}(\vec{x}) = \vec{w} \cdot \vec{x} + b
$$

损失函数: 

$$
\text{MSE} = \frac{1}{2m} \sum_{i=1}^{m} \left( y^{(i)} - \hat{y}^{(i)} \right)^2
$$

梯度下降：

分别对每个w和b求偏导数，然后更新w和b
 

In [5]:
# 定义损失函数
def compute_loss(w, b):
    return np.sum((y - (np.dot(w, x.T) + b)) ** 2) / (2 * m)   # w 1x3 x.T 3x10 y 1x10 y-np.dot(w, x.T) 1x10 sum=number

# 定义梯度下降
def gradient_descent(w, b, alpha, num_iter):
    for _ in range(num_iter):
        error = y - np.dot(w, x.T) - b # 1x10
        # 计算梯度
        dw = -np.dot(x.T , error) / m  # dw 1x3 
        db = -np.sum(error) / m  # db 1x1
        # 更新w和b
        w -= alpha * dw
        b -= alpha * db
    return w, b

In [6]:
# 主函数
if __name__ == "__main__":
    # 初始化w和b
    w = np.zeros(x.shape[1])
    b = 0.0
    # 设置学习率
    alpha = 0.01
    # 设置迭代次数
    num_iter = 1000
    # 进行梯度下降
    w,b = gradient_descent(w,b,alpha,num_iter)
    print("w:", w)
    print("b:", b)
    # 计算损失
    loss = compute_loss(w,b)
    print("loss:", loss)

w: [0.30933908 0.51500581 0.17832512]
b: -3.2582270215186813e-16
loss: 0.0027531700168465624


# 编码中遇到的错误

梯度下降算法中，把x.T和error相乘了，正确应使用矩阵乘法。